# Introduction to PINNs
### The static, linear Euler-Bernoulli beam

---

Lecture: "Physics-augmented machine learning" @ Cyber-Physical Simulation, TU Darmstadt

Lecturer: Prof. Oliver Weeger

Author: Jasper O. Schommartz

---

#### In this notebook, you will...

* learn to explain the fundamental structure of PINNs.
* calibrate a PINN for the linear Euler-Bernoulli cantilever beam.
* modify boundary conditions in PINN calibration.
* experience the importance of non-dimensionalization on PINN calibration.

In [ ]:
# Run this cell if you are using Google Colab
!git clone https://github.com/CPSHub/LecturePhysicsAwareML.git
%cd LecturePhysicsAwareML/PINNs
%pip install -e .

In [ ]:
# Run this cell if you are working locally
%cd ..
%pip install -e .

## 1. Theory

### 1.1 Linear Euler-Bernoulli Beam

In this task we consider a linear Euler-Bernoulli beam (LEBB).
The mechanical behavior of the linear beam is decribed by a fourth order differential equation, which reads

$$
\begin{align}
    EI \partial_{x^4} w = q(x).
\end{align}
$$

Here, $w$ is the vertical displacement and $q(x)$ denotes a transversal line load depending on the horizontal location $x$. The Young's modulus and the moment of inertia are denoted by $E$ and $I$, respectively. For the scope of this work we **consider only constant line loads**, i.e, $q(x) = q_0$.


<img src="../img/beam.png" alt="Beam Image" width="600">

For the cantilever beam shown in the image above. The boundary conditions read

$$
w(0) = w'(0) = 0
$$

and the Neumann boundary conditions read

$$
Q(L) = F,\quad M(L) = 0.
$$

### 1.2 Non-dimensionalization

It has been reported in literature, that non-dimensionlizing PDEs before PINN calibration can improve convergence [[1][Wang2023]]. For the LEBB, problems with dimensionality arise depending on the choise of Young's modulus $E$, moment of inertia $L$ and beam length $L$.

Equation (1) may be non-dimentionalized as follows: First we define some dependent non-dimensional variables $\tilde{x}$, $\tilde{w}$, and $\tilde{q}$, which are realated to their dimensional counter parts as

$$
\begin{align}
x = x_0 \tilde{x}, \quad w = w_0 \tilde{w},\quad q = q_0 \tilde{q} \tag{2}
\end{align}
$$

The derivatives w.r.t. $w$ consequently follow as

$$
\begin{align}
\partial_x w = \frac{w_0}{x_0} \partial_{\tilde{x}} \tilde{w},\quad \partial_{x^2} w = \frac{w_0}{x_0^2} \partial_{\tilde{x}^2} \tilde{w},\quad \partial_{x^3} w = \frac{w_0}{x_0^3} \partial_{\tilde{x}^3} \tilde{w},\quad \partial_{x^4} w = \frac{w_0}{x_0^4} \partial_{\tilde{x}^4} \tilde{w}. \tag{3}
\end{align}
$$

By inserting Equations (2) and (3) into (1) assuming a constant line load and doing some refomulation we obtain

$$
\partial_{\tilde{x}^4} \tilde{w} = \frac{x_0^4}{EI w_0} q_0.
$$

From this we can choose $x_0$, $w_0$ and $q_0$ in different ways. First we set $x_0 = L$, which seems like a reasonable choice. If there is no line loas, we choose $q_0 = 1$. Otherwise $q_0$ takes the value of the constant line load. In order to set all factors to 1, we choose

$$
w_0 = \frac{q_0 L^4}{EI}
$$

With this, the bending moment $M = -EI \partial_{x^2} w$ and the transversal force $Q = -EI \partial_{x^3} w$ may be non-dimensionalized as

$$
\tilde{M} = \frac{M}{q_0 L^2}, \quad \text{and} \quad \tilde{Q} = \frac{Q}{q_0 x_0}
$$


### 1.3 PINN model

<img src="../img/pinn_lebb.png" alt="PINN LEBB" width=600>

The PINN architecture shown above is used in this task. Thereby a feed forward neural network (FFNN) is used to map from a collocation point $x$ to the displacement $w$. The PINN is calibrated by minimizing the residual of Equation (1) and well as the related boundary conditions. Hence, the physics is introduced in a weak manner. Futhermore, the PINN does not require any data for calibration, as the PDE solution is uniquely characterized by Equation (1) and the boudary condition.

The loss function consists of three parts. It reads

$$
\mathcal{L} = \mathcal{L}_{\text{data}} + \mathcal{L}_{\text{res}} + \mathcal{L}_{\text{BCs}}
$$

Here, $\mathcal{L}_{\text{data}}$, denotes the error on a set of labeled data, $\mathcal{L}_{\text{res}}$ denotes the residual of Equation (1), and $\mathcal{L}_{\text{BCs}}$ denotes the residual of the Dirichlet and Neumann boundary conditions. **In this simple task we do not consider any data loss**. In pratice, weighting of the different loss terms can be an issue, especially for high-dimensional problems. Therefore, different methods for loss and gradient weighting have been developed [[1][Wang2023]], which are out of the scope of this tutorial.

[Wang2023]: https://arxiv.org/abs/2308.08468

## 2. PINN Calibration
### 2.1 Definition of Boundary Conditions

You can use this section to define arbitrary boundary conditions on the LEBB.

In [ ]:
import jax.numpy as jnp
from paml_pinns.lebb import Config, get_data_decorator


@get_data_decorator
def get_data(config: Config):
    if config.bc_case == 0:
        w_bc_coords = jnp.array([0.0])
        w_bc_values = jnp.array([0.0])
        w_x_bc_coords = jnp.array([0.0])
        w_x_bc_values = jnp.array([0.0])
        M_bc_coords = jnp.array([config.L])
        M_bc_values = jnp.array([0.0])
        Q_bc_coords = jnp.array([config.L])
        Q_bc_values = jnp.array([config.F])
    elif config.bc_case == 1:
        # Start of task scope
        ######################
        pass  # remove this for testing
        # w_bc_coords =
        # w_bc_values =
        # w_x_bc_coords =
        # w_x_bc_values =
        # M_bc_coords =
        # M_bc_values =
        # Q_bc_coords =
        # Q_bc_values =
        ######################
        # End of task scope
    else:
        NotImplementedError(
            "Data generation is not implemented for these boundary conditions."
        )

    bc = {
        "w_bc_coords": w_bc_coords,
        "w_bc_values": w_bc_values,
        "w_x_bc_coords": w_x_bc_coords,
        "w_x_bc_values": w_x_bc_values,
        "M_bc_coords": M_bc_coords,
        "M_bc_values": M_bc_values,
        "Q_bc_coords": Q_bc_coords,
        "Q_bc_values": Q_bc_values,
    }
    return bc


### 2.2 Configuration of Beam Properties

In this section, the properties of the beam can be defined for the boundary condition cases above. Note that line loads are considered to be constant.

In [ ]:
from paml_pinns.lebb import get_config_decorator
from typing import Tuple


@get_config_decorator
def get_config(bc_case: int) -> Tuple[float, float, float, float]:
    if bc_case == 0:
        EI = 1e6
        L = 1.0
        F = 1.0
        q = 0.0
    elif bc_case == 1:
        EI = 1e6
        L = 1.0
        F = 0.0  # Unused dummy
        q = 1.0
    else:
        raise NotImplementedError(
            "No configuration implemented for these boundary conditions."
        )

    return EI, L, F, q

### 2.3 Data Generation and Model Creation

Here, we generate input and solution data for the LEBB. This is only possible, since Equation (1) can be solved analytically. However, **note** that solution data is not required for PINN calibration. Next, and instance of the PINN model is create using a pseudo-random seed.

In [ ]:
import jax
from paml_pinns.lebb import PINN


config = get_config(bc_case=0, non_dim=False)
x, y, bc, EI, L, q = get_data(config)
weights = {"w_bc": 1.0, "w_x_bc": 1.0, "M_bc": 1.0, "Q_bc": 1.0, "rw": 1.0}

key = jax.random.PRNGKey(1234)
model = PINN(EI, L, q, bc, key=key)


### 2.4 Model Calibration and Evaluation

The model is calibrated and evaluated. **Note**, that the solution data `y` is not passed to the `train` function. Hence, the PINN is trained exclusively using residuals.

In [ ]:
from paml_pinns import train
from paml_pinns.lebb import evaluate


model = train(model, x, weights, steps=50_000)
evaluate(model, x, y)

## 3 Tasks

### Task 1: Effect of Non-Dimensionalization

For `bc_case=0`, ...

a) Set `EI=1e6`, and `non_dim=False`. Can the model approximate the solution?

b) Now set the `EI = 1.0`. Can the model approximate the solution now?

c) Set `EI=1e6` again and set `non_dim=True`. What different to you observe compared to a)? Explain.  

### Task 2: Implementation of Boundary Conditions

Implement the boundary conditions for `bc_case=1` within `get_data()`. Next, calibrate the PINN and test your implementation against the analytical solution. Note, boundary conditions can be set equal to `None`to be ignored.

*Solution proposal:*

In [ ]:
import jax.numpy as jnp

from paml_pinns.lebb import Config, get_data_decorator


@get_data_decorator
def get_data(config: Config):
    if config.bc_case == 1:
        w_bc_coords = jnp.array([0.0, config.L])
        w_bc_values = jnp.array([0.0, 0.0])
        w_x_bc_coords = None
        w_x_bc_values = None
        M_bc_coords = jnp.array([0.0, config.L])
        M_bc_values = jnp.array([0.0, 0.0])
        Q_bc_coords = None
        Q_bc_values = None
    else:
        NotImplementedError(
            "Data generation is not implemented for these boundary conditions."
        )

    bc = {
        "w_bc_coords": w_bc_coords,
        "w_bc_values": w_bc_values,
        "w_x_bc_coords": w_x_bc_coords,
        "w_x_bc_values": w_x_bc_values,
        "M_bc_coords": M_bc_coords,
        "M_bc_values": M_bc_values,
        "Q_bc_coords": Q_bc_coords,
        "Q_bc_values": Q_bc_values,
    }
    return bc